# T1 T2 T3 Tis 类生成 patch

In [8]:
import glob
import os
import numpy as np
import openslide
import cv2
from tqdm import tqdm

dir_list = ['T1', 'T2', 'T3', 'Tis']
level = 2
edge = 512

for temp in dir_list:
    wsi_path = f'E:/3rdSEED/svs/{temp}'
    patch_path = f'D:/AI/3rdSEED/SEED2022_gastric_cancer_classification/data/{temp}_patch'
    ann_path = f'D:/AI/3rdSEED/SEED2022_gastric_cancer_classification/npy/{temp}'
    wsi_name_list = os.listdir(ann_path)
    
    print(f"processing {temp} .......")

    for i in range(len(wsi_name_list)):
        wsi_name_list[i] = wsi_name_list[i] + '.svs'

    for item in tqdm(wsi_name_list):      # item 是svs的文件名
        slide = openslide.OpenSlide(os.path.join(wsi_path, item))  # 获取 svs 图片
        down_sample_rate = int(slide.level_downsamples[level])
        judge_edge = int(edge/down_sample_rate)
        
        drop_suffix = item.split('.')[0]
        save_path = os.path.join(patch_path, drop_suffix)
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        npy_path = glob.glob(os.path.join(ann_path, drop_suffix, '*.npy'))
        region_num = len(npy_path)
        region_points = []
        # 一次取一个标注区域，在该区域中取 50 个 patch
        anno_num = 0
        for it in npy_path:
            region_points = [(np.load(it)/slide.level_downsamples[level]).astype(np.int32)]
            region_points = np.array(region_points)

            W, H = slide.level_dimensions[level]
            mask = np.zeros((H, W))
            cv2.fillPoly(mask, region_points, 255)
            x, y = np.where(mask)
            all_point = np.stack(np.vstack((x, y)), axis=1)
            sample_num = 500
            if all_point.shape[0] > sample_num:
                sample_point = all_point[np.random.randint(all_point.shape[0], size=sample_num), :]
            else:
                sample_point = all_point
            save_num = 0
            for idx, (x, y) in enumerate(sample_point):
                # print('-------------------------')
                # print(f'{drop_suffix}_Anno{anno_num}_{idx}')
                # print('edge',edge)
                # print('y',y)
                # print('W',W)
                # print('x',x)
                # print('H',H)
                # print('-------------------------')
                if save_num < 40:
                    if y + edge/2 > W or x + edge/2 > H or mask[x+judge_edge, y] == 0 or mask[x, y+judge_edge] == 0 or mask[x+judge_edge, y+judge_edge] == 0:
                        continue
                    else:
                        img = slide.read_region((int(np.max((y-edge/2)*down_sample_rate,0)), int(np.max((x-edge/2)*down_sample_rate,0))), level, (edge, edge))
                        # 有色区域大于一定比例的图片保留
                        img_gray = cv2.cvtColor(np.array(img)[:, :, :3].astype(np.uint8), cv2.COLOR_BGR2GRAY)
                        ret, thresh = cv2.threshold(img_gray, 200, 255, cv2.THRESH_BINARY_INV)
                        # cv2.imshow('thresh',thresh)
                        # cv2.waitKey(0)
                        # print(len(np.where(thresh)[0]))
                        # print('0.2*H*W',0.2*H*W)
                        if(len(np.where(thresh)[0])>0.1*edge*edge):
                            img.save(os.path.join(save_path, f'{drop_suffix}_Anno{anno_num}_{idx}.png'))
                            save_num += 1
                else:
                    break
                            
            anno_num += 1


processing T1 .......


100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [02:33<00:00,  9.05s/it]


processing T2 .......


100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [02:40<00:00,  6.17s/it]


processing T3 .......


100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [01:45<00:00,  4.22s/it]


processing Tis .......


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [03:04<00:00,  6.14s/it]


# T0 类生成 patch

In [2]:
import os
import openslide
import random
from tqdm import tqdm
import cv2
import numpy as np

path = r'E:/3rdSEED/svs/T0'
svs_file = os.listdir(path)
save = r'D:/AI/3rdSEED/SEED2022_gastric_cancer_classification/data/T0_patch'

level = 2
edge = 512

# 读取到边界时，将边界以外的透明区域转换为白色。
def transparence2white(img):  
    sp = img.shape  # 获取图片维度
    width = sp[0]  # 宽度
    height = sp[1]  # 高度
    for yh in range(height):
        for xw in range(width):
            color_d = img[xw, yh]  # 遍历图像每一个点，获取到每个点4通道的颜色数据
            if (color_d[3] == 0):  # 最后一个通道为透明度，如果其值为0，即图像是透明
                img[xw, yh] = [255, 255, 255, 255]  # 则将当前点的颜色设置为白色，且图像设置为不透明
    return img

for svs in tqdm(svs_file):
    svs_drop_f = svs.split('.')[0]
    svs_path = os.path.join(path, svs)
    slide = openslide.OpenSlide(svs_path)
    if len(slide.level_downsamples)<=level:
        level = len(slide.level_downsamples)-1
    else:
        level = 2
    down_sample_rate = int(slide.level_downsamples[level])
    judge_edge = int(edge/down_sample_rate)
    
    #print("kk", svs_drop_f)
    H, W = slide.level_dimensions[level]
    #print(H, W)
    list_x = [i for i in range(H)]
    list_y = [i for i in range(H)]
    random.shuffle(list_x)
    random.shuffle(list_y)
    save_path = os.path.join(save, svs_drop_f)
    #print('debug')
    #print(svs_drop_f)
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    save_num = 0
    for idx in range(500):
        if save_num<40:
            if list_x[idx] + judge_edge > H or list_y[idx] + judge_edge> W:
                continue
            else:
                img = slide.read_region((list_x[idx]*down_sample_rate, list_y[idx]*down_sample_rate), level, (edge, edge))
                # 有色区域大于一定比例的图片保留
                np_img = np.array(img)[:, :, :4].astype(np.uint8)
                transparence2white(np_img)
                np_img = np_img[:, :, :3]
                img_gray = cv2.cvtColor(np_img, cv2.COLOR_BGR2GRAY)
                ret, thresh = cv2.threshold(img_gray, 200, 255, cv2.THRESH_BINARY_INV)
                # cv2.imshow('img_gray',img_gray)
                # cv2.imshow('thresh',thresh)
                # cv2.waitKey(0)
                # print(len(np.where(thresh)[0]))
                # print('0.1*edge*edge',0.1*edge*edge)
                if(len(np.where(thresh)[0])>0.1*edge*edge):
                    img.save(os.path.join(save_path, f'{svs_drop_f}_{save_num}.png'))
                    save_num += 1
        else:
            break

100%|██████████████████████████████████████████████████████████████████████████████| 200/200 [1:16:16<00:00, 22.88s/it]


# Test 类 生成 patch

In [19]:
import os
import openslide
import numpy as np
import cv2
from tqdm import tqdm


svs_path = r'E:/3rdSEED/svs/Test'
root_path = r'D:/AI/3rdSEED/SEED2022_gastric_cancer_classification/npy/Test'
save_p = r'D:/AI/3rdSEED/SEED2022_gastric_cancer_classification/Test_patch'

level = 2
edge = 512
sample_num = 500

file_list = os.listdir(root_path)
for file_name in tqdm(file_list):
    file_path = os.path.join(root_path, file_name)      # F:\svs\含Test标注的npy\4mF7tL
    annotation_list = os.listdir(file_path)
    for annotation_name in annotation_list:
        annotation_path = os.path.join(file_path, annotation_name)
        save_path = os.path.join(save_p, file_name, annotation_name.split('.')[0].replace(' ', ''))
        if not os.path.exists(save_path):
            os.makedirs(save_path)

        slide = openslide.OpenSlide(os.path.join(svs_path, f'{file_name}.svs'))
        down_sample_rate = int(slide.level_downsamples[level])
        judge_edge = int(edge/down_sample_rate)
        
        region_points = (np.load(annotation_path)/down_sample_rate).astype(np.int32)
        
        W, H = slide.level_dimensions[level]
        mask = np.zeros((H, W))
        cv2.fillConvexPoly(mask, region_points, 255)
        x, y = np.where(mask)
        all_point = np.stack(np.vstack((x, y)), axis=1)
        if all_point.shape[0] == 0:
            print(annotation_name)
            continue
        sample_point = all_point[np.random.randint(all_point.shape[0], size=sample_num), :]
        save_num = 0
        for idx, (x, y) in enumerate(sample_point):
            if save_num<40:
                if y + judge_edge > W or x + judge_edge > H or \
                mask[int(max(x-edge/8,0)), y] == 0 or \
                mask[x, int(max(y-edge/8,0))] == 0 or \
                mask[int(max(x-edge/8,0)), int(max(y-edge/8,0))] == 0 or\
                mask[int(min(x+edge/8,H-1)), y] == 0 or \
                mask[x, int(min(y+edge/8,W-1))] == 0 or \
                mask[int(min(x+edge/8,H-1)), int(min(y+edge/8,W-1))] == 0:
                    continue
                else:
                    img = slide.read_region((int(np.max((y-edge/2)*down_sample_rate,0)), int(np.max((x-edge/2)*down_sample_rate,0))), level, (edge, edge))
                    # 有色区域大于一定比例的图片保留
                    np_img = np.array(img)[:, :, :4].astype(np.uint8)
                    transparence2white(np_img)
                    np_img = np_img[:, :, :3]
                    img_gray = cv2.cvtColor(np_img, cv2.COLOR_BGR2GRAY)
                    ret, thresh = cv2.threshold(img_gray, 200, 255, cv2.THRESH_BINARY_INV)
                    if(len(np.where(thresh)[0])>0.1*edge*edge):
                        img.save(os.path.join(save_path, f'{save_num}.png'))
                        save_num += 1
            else:
                break


100%|██████████████████████████████████████████████████████████████████████████████████| 42/42 [26:07<00:00, 37.33s/it]


# 文件夹合并

合并前先删去data中的npy文件夹

In [20]:
import os
import glob
from PIL import Image
from tqdm import tqdm

sour_path = r'D:/AI/3rdSEED/SEED2022_gastric_cancer_classification/data'
save_path = ['D:/AI/3rdSEED/SEED2022_gastric_cancer_classification/raw/T0',
             'D:/AI/3rdSEED/SEED2022_gastric_cancer_classification/raw/T1', 
             'D:/AI/3rdSEED/SEED2022_gastric_cancer_classification/raw/T2', 
             'D:/AI/3rdSEED/SEED2022_gastric_cancer_classification/raw/T3', 
             'D:/AI/3rdSEED/SEED2022_gastric_cancer_classification/raw/Tis']
t_patch_path = os.listdir(sour_path)
for idx, t_path in enumerate(t_patch_path):
    print('processing ',t_path)
    if not os.path.exists(save_path[idx]):
        os.makedirs(save_path[idx])
    json_path = os.path.join(sour_path, t_path)
    json_list = os.listdir(json_path)
    for item in tqdm(json_list):
        temp = os.path.join(json_path, item, '*.png')
        img_path = glob.glob(temp)
        for j, i in enumerate(img_path):
            img = Image.open(i)
            img.save(os.path.join(save_path[idx], f'{item}_{j}.png'))

processing  T0_patch


100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [08:08<00:00,  2.44s/it]


processing  T1_patch


100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [02:01<00:00,  7.17s/it]


processing  T2_patch


100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [02:12<00:00,  5.08s/it]


processing  T3_patch


100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [01:41<00:00,  4.05s/it]


processing  Tis_patch


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [02:33<00:00,  5.12s/it]
